In [1]:
import tensorflow as tf
x=tf.Variable(3,name='x')
y=tf.Variable(4, name='y')
f=x*x*y+y+2
init=tf.global_variables_initializer()


In [2]:
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
print(result)
sess.close()

42


In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()

In [9]:
with tf.Session() as sess:
    init.run()
    result=f.eval()
    print(result)

42


# Linear regression on California housing dataset

## Normal equations:

In [2]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing()
m,n=housing.data.shape

housing_data_plus_bias=np.c_[np.ones((m,1)), housing.data]

X=tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT=tf.transpose(X)

theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_value=theta.eval()

print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


## Gradient descent

Need to scale the data first for better convergence

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data=scaler.fit_transform(housing.data)

scaled_housing_data_plus_bias=np.c_[np.ones((m,1)), scaled_housing_data]


In [17]:
nloops=1000
learning_rate=0.01

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name='y')
theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')
y_pred=tf.matmul(X,theta, name='prediction')
error=y_pred-y
mse=tf.reduce_mean(tf.square(error, name='mse'))

gradients=2/m*tf.matmul(tf.transpose(X),error)

training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        if loop%100==0:
            print('loop', loop, 'MSE=', mse.eval())
        sess.run(training_op)
        
    best_theta=theta.eval()
    
    print(best_theta)

loop 0 MSE= 4.85334
loop 100 MSE= 0.747335
loop 200 MSE= 0.668665
loop 300 MSE= 0.639556
loop 400 MSE= 0.617165
loop 500 MSE= 0.599403
loop 600 MSE= 0.585237
loop 700 MSE= 0.573893
loop 800 MSE= 0.564771
loop 900 MSE= 0.557409
[[ 2.06855249]
 [ 0.97460502]
 [ 0.16666   ]
 [-0.50069165]
 [ 0.48311004]
 [ 0.01104329]
 [-0.04647031]
 [-0.4144997 ]
 [-0.40015447]]


using autodiff:

In [18]:
nloops=1000
learning_rate=0.01

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name='y')
theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')
y_pred=tf.matmul(X,theta, name='prediction')
error=y_pred-y
mse=tf.reduce_mean(tf.square(error, name='mse'))

#difference is here
gradients=tf.gradients(mse,[theta])[0]

training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        if loop%100==0:
            print('loop', loop, 'MSE=', mse.eval())
        sess.run(training_op)
        
    best_theta=theta.eval()
    
    print(best_theta)

loop 0 MSE= 10.9403
loop 100 MSE= 0.73624
loop 200 MSE= 0.533643
loop 300 MSE= 0.526731
loop 400 MSE= 0.525957
loop 500 MSE= 0.525583
loop 600 MSE= 0.52531
loop 700 MSE= 0.525099
loop 800 MSE= 0.524938
loop 900 MSE= 0.524811
[[ 2.06855226]
 [ 0.84255821]
 [ 0.12524015]
 [-0.28237295]
 [ 0.31628352]
 [-0.00231135]
 [-0.04015082]
 [-0.84042633]
 [-0.81221974]]


Using an optimizer

In [7]:
nloops=5000
learning_rate=0.01

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name='y')
theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')
y_pred=tf.matmul(X,theta, name='prediction')
error=y_pred-y
mse=tf.reduce_mean(tf.square(error, name='mse'))

#use an optimizer
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        if loop%500==0:
            print('loop', loop, 'MSE=', mse.eval())
        sess.run(training_op)
        
    best_theta=theta.eval()
    
    print(best_theta)

loop 0 MSE= 3.26923
loop 500 MSE= 0.561195
loop 1000 MSE= 0.537394
loop 1500 MSE= 0.529233
loop 2000 MSE= 0.526227
loop 2500 MSE= 0.525074
loop 3000 MSE= 0.524621
loop 3500 MSE= 0.524441
loop 4000 MSE= 0.524369
loop 4500 MSE= 0.52434
[[ 2.06855249]
 [ 0.83274233]
 [ 0.11935294]
 [-0.27139792]
 [ 0.31053811]
 [-0.0043248 ]
 [-0.03944373]
 [-0.8925699 ]
 [-0.86358738]]


## Mini-batch Gradient descent

In [6]:
A=tf.placeholder(tf.float32, shape=(None,3))
B=A+5
with tf.Session() as sess:
    B_val_1=B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2=B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})

    
print(B_val_1)
print(B_val_2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [97]:
nloops=100
learning_rate=0.01

import pandas as pd
df=pd.DataFrame(np.c_[housing.target, scaled_housing_data], columns=['y']+housing.feature_names)
df['Bias']=np.ones((m,1))

cols=list(df.columns.values)
cols_temp=cols[1:len(cols)-1]

cols[1]='Bias'
cols[2:]=cols_temp

df=df[cols]


df.to_csv('data.csv',)

batch_size=100
n_batches=int(np.ceil(m/batch_size))


X=tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y=tf.placeholder(tf.float32, shape=(None, 1), name='y')

theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')
y_pred=tf.matmul(X,theta, name='prediction')
error=y_pred-y
mse=tf.reduce_mean(tf.square(error, name='mse'))

#use an optimizer
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        for batch_index in range(n_batches):
            data_batch=df.iloc[batch_index*batch_size: min((batch_index+1)*batch_size, m)]
            y_batch=np.asarray(data_batch.y).reshape((-1,1))
            X_batch=np.asarray(data_batch[data_batch.columns[1:]])
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if loop%10==0:
            print('loop', loop, 'MSE=', mse.eval(feed_dict={X: X_batch, y: y_batch }))
    
    best_theta=theta.eval()
    
    print(best_theta)
    
    


loop 0 MSE= 0.399021
loop 10 MSE= 0.124524
loop 20 MSE= 0.124451
loop 30 MSE= 0.124403
loop 40 MSE= 0.124391
loop 50 MSE= 0.124389
loop 60 MSE= 0.124388
loop 70 MSE= 0.124388
loop 80 MSE= 0.124388
loop 90 MSE= 0.124388
[[ 2.00290895]
 [ 0.78347152]
 [ 0.13612019]
 [-0.23013733]
 [ 0.27156752]
 [-0.00706378]
 [-0.01321162]
 [-0.86900991]
 [-0.89599925]]


['y',
 'Bias',
 'MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [91]:
cols

Index(['y', 'MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population',
       'AveOccup', 'Latitude', 'Longitude', 'Bias'],
      dtype='object')